In [1]:
import os
import datetime
from pathlib import Path
from collections import defaultdict
import scipy
import random
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
pd.set_option('display.max_colwidth',100)
import joblib
import pickle
import csv

import sklearn           
from sklearn.metrics import make_scorer
import xgboost as xgb     
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

import mCDR_utils as prerun

import gcsfs
fs = gcsfs.GCSFileSystem()

2025-04-23 23:48:13.046512: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-23 23:48:13.061398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-23 23:48:13.079760: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-23 23:48:13.085111: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-23 23:48:13.098800: I tensorflow/core/platform/cpu_feature_guar

In [2]:
approach = 'nmse' 
#approach = 'bias' 

recon_output_dir = 'gs://leap-persistent/your_username'
hyperparam_dir = 'gs://leap-persistent/your_username'
ml_model_save_path = '/home/jovyan/mCDR'
path_seeds = 'gs://leap-persistent/abbysh/pickles/random_seeds.npy'

jobs = 30

# For bias as hyperparameter error metric

In [3]:
def bias_fxn(truth,pred):
    bias = pred - truth
    return np.abs(np.nanmean(bias))

In [4]:
bias_scorer = make_scorer(bias_fxn, greater_is_better=False)

In [3]:
with fs.open('gs://leap-persistent/abbysh/pickles/random_seeds.npy') as pam:
     random_seeds = np.load(pam)

In [4]:
date_range_start = '2020-02-01T00:00:00.000000000'
date_range_end = '2022-01-01T00:00:00.000000000'

dates = pd.date_range(start=date_range_start, 
                      end=date_range_end,freq='MS') + np.timedelta64(14, 'D')

init_date = str(dates[0].year) + format(dates[0].month,'02d')
fin_date = str(dates[-1].year) + format(dates[-1].month,'02d')

In [5]:
val_prop = .2
test_prop = .2


# xg_param_grid = {"n_estimators":[500, 1000, 2000, 4000],
#                  "max_depth":[5, 10, 15],
#                  "learning_rate":[0.10, 0.30, 0.40]
#                 }

best_params = {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 4000}

# Results from previous bias grid search
#best_params = {'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 2000}


features_sel = ['sst','sst_anom','sss','sss_anom','mld_clim_log','chl_log','chl_log_anom','xco2','A', 'B', 'C', 'T0', 'T1'] 
target_sel = ['pco2_residual'] 

In [6]:
select_dates = []
test_dates = []

for i in range(0,len(dates)):
    
    if i % 5 != 0:
        select_dates.append(dates[i]) 
    if i % 5 == 0:
        test_dates.append(dates[i]) 
        

year_mon = []

for i in range(0,len(select_dates)):
    
    tmp = select_dates[i]
    year_mon.append(f"{tmp.year}-{tmp.month}")
    
test_year_mon = []

for i in range(0,len(test_dates)):
    
    tmp = test_dates[i]
    test_year_mon.append(f"{tmp.year}-{tmp.month}")

In [7]:
K_folds = 3
first_mem = False # Change to true if applying grid search

metrics_output_dir = '/home/jovyan/mCDR'

test_perform_fname = f"{metrics_output_dir}/{approach}_test_performance_2020-2022.csv"
unseen_perform_fname = f"{metrics_output_dir}/{approach}_unseen_performance_2020-2022.csv"

print(datetime.datetime.now())


file_path = 'gs://leap-persistent/abbysh/ncar_files/post01_xgb_inputs/0022_north/MLinput_CESM_mon_.1x.1_202002_202201.pkl'
      
with fs.open(file_path,'rb') as filee:
    df = pd.read_pickle(filee)
    df['year'] = df.index.get_level_values('time').year
    df['mon'] = df.index.get_level_values('time').month
    df['year_month'] = df['year'].astype(str) + "-" + df['mon'].astype(str)
    recon_sel = (~df[features_sel+target_sel].isna().any(axis=1)) & ((df[target_sel] < 250) & (df[target_sel] > -250)).to_numpy().ravel()

    sel = (recon_sel & ((df['socat_mask'] == 1)))
    train_sel = (sel & (pd.Series(df['year_month']).isin(year_mon))).to_numpy().ravel()
    print(sum(train_sel)) 

    test_sel = (sel & (pd.Series(df['year_month']).isin(test_year_mon))).to_numpy().ravel()
    print(sum(test_sel))
    unseen_sel = (recon_sel & (df['socat_mask'] == 0))

    X = df.loc[sel,features_sel].to_numpy()
    y = df.loc[sel,target_sel].to_numpy().ravel()
         
    Xtrain = df.loc[train_sel,features_sel].to_numpy()             
    ytrain = df.loc[train_sel,target_sel].to_numpy().ravel()

    N = Xtrain.shape[0]
    train_val_idx, train_idx, val_idx, test_idx = prerun.train_val_test_split(N, test_prop, val_prop, random_seeds, seed_loc)
    X_train_val, X_train, X_val, X_test_tmp, y_train_val, y_train, y_val, y_test_tmp = prerun.apply_splits(Xtrain, ytrain, train_val_idx, train_idx, val_idx, test_idx) 

    X_test = df.loc[test_sel,features_sel].to_numpy()                
    y_test = df.loc[test_sel,target_sel].to_numpy().ravel()    
   
    if first_mem:
        model = XGBRegressor(random_state=random_seeds[4,seed_loc], n_jobs=jobs)
        param_grid = xg_param_grid
        if approach == 'bias':
            grid = GridSearchCV(model, param_grid, scoring=bias_scorer, cv=K_folds, return_train_score=True, refit=False, 
                            verbose=3)
        elif approach == 'nmse':
            grid = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=K_folds, return_train_score=True, refit=False, 
                            verbose=3)
        print(grid)
        grid.fit(X_train_val, y_train_val)
        best_params = grid.best_params_
        print(best_params)
        scores = pd.DataFrame(grid.cv_results_)

        scores.to_csv(f'{hyperparam_dir}/CVgrid_scores_{approach}.csv')
    
    train_performance = defaultdict(dict)
    test_performance = defaultdict(dict)
    unseen_performance = defaultdict(dict)

    model = XGBRegressor(random_state=random_seeds[5,seed_loc], **best_params, n_jobs=jobs)
    model.fit(X_train_val, y_train_val)          

    prerun.save_model(model, dates, ml_model_save_path, approach)   
    
    y_pred_test = model.predict(X_test)
    test_performance = prerun.evaluate_test(y_test, y_pred_test)
    
    fields = test_performance.keys()
    test_row_dict = dict()
    
    for field in fields:
        test_row_dict[field] = test_performance[field]

    file_exists = os.path.isfile(test_perform_fname)
    
    with open(test_perform_fname, 'a') as f_object:
        writer = csv.DictWriter(f_object, fieldnames = test_row_dict.keys())
        if not file_exists:
            writer.writeheader() 
        writer.writerow(test_row_dict)
        
    print('test performance metrics:', test_performance)


    y_pred_unseen = model.predict(df.loc[unseen_sel,features_sel].to_numpy())
    y_unseen = df.loc[unseen_sel,target_sel].to_numpy().ravel()
    unseen_performance = prerun.evaluate_test(y_unseen, y_pred_unseen)
    
    fields = unseen_performance.keys()
    unseen_row_dict = dict()
    
    for field in fields:
        unseen_row_dict[field] = unseen_performance[field]

    file_exists = os.path.isfile(unseen_perform_fname)
    
    with open(unseen_perform_fname, 'a') as f_object:
        writer = csv.DictWriter(f_object, fieldnames = unseen_row_dict.keys())
        if not file_exists:
            writer.writeheader() 
        writer.writerow(unseen_row_dict)
    
    print('unseen performance metrics:', unseen_performance)

    y_pred_seen = model.predict(X)

    df['pco2_residual_full_recon'] = np.nan
    df.loc[unseen_sel,['pco2_residual_full_recon']] = y_pred_unseen   
    df.loc[sel,['pco2_residual_full_recon']] = y_pred_seen
    
    df['pco2_residual_test'] = np.nan
    df.loc[unseen_sel,['pco2_residual_test']] = np.nan
    df.loc[sel,['pco2_residual_test']] = y_pred_seen
    df.loc[train_sel, ['pco2_residual_test']] = np.nan

    df['pco2_residual_train'] = np.nan
    df.loc[unseen_sel,['pco2_residual_train']] = np.nan
    df.loc[sel,['pco2_residual_train']] = y_pred_seen
    df.loc[test_sel, ['pco2_residual_train']] = np.nan

    df['pco2_residual_unseen'] = np.nan
    df.loc[unseen_sel,['pco2_residual_unseen']] = y_pred_unseen
    df.loc[sel,['pco2_residual_unseen']] = np.nan

    df['pco2_residual_seen'] = np.nan
    df.loc[unseen_sel,['pco2_residual_seen']] = np.nan
    df.loc[sel,['pco2_residual_seen']] = y_pred_seen

    DS_recon = df[['socat_mask','pco2_residual','pco2_residual_full_recon','pco2_residual_seen','pco2_residual_unseen', 'pco2_residual_test', 'pco2_residual_train']].to_xarray()

    prerun.save_recon(DS_recon, dates, recon_output_dir, approach)   
            
print('end of all members', datetime.datetime.now())
print('saving performance metrics')

2025-04-23 23:48:28.819155
481939
120159
Starting model saving process
Save complete
test performance metrics: {'mse': 243.11351013183594, 'mae': 10.761034965515137, 'medae': 7.1709595, 'max_error': 92.95462, 'bias': 4.045164, 'r2': 0.8906974792480469, 'corr': 0.9542875391108607, 'cent_rmse': 15.058226080380626, 'stdev': 50.301888, 'amp_ratio': 0.9910926, 'stdev_ref': 47.161705, 'range_ref': 330.80823, 'iqr_ref': 74.75595092773438}
unseen performance metrics: {'mse': 2197.632080078125, 'mae': 35.79771041870117, 'medae': 27.717058, 'max_error': 331.72897, 'bias': -3.0952494, 'r2': -2.0441248416900635, 'corr': 0.17478747793176985, 'cent_rmse': 46.77657530691109, 'stdev': 43.273197, 'amp_ratio': 0.74493426, 'stdev_ref': 26.868677, 'range_ref': 499.95062, 'iqr_ref': 23.909423828125}
Starting reconstruction saving process
gs://leap-persistent/abbysh/ncar_files/post02_xgb_output/0022_north/reconstructions/recon_fC02residual_nmse_mon_01x01_202002_202201.zarr
Save complete
end of all members 2

/srv/conda/envs/notebook/lib/python3.12/site-packages/zarr/api/asynchronous.py:197: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


## Add pCO2-T back

In [ ]:
regridded_members_dir = 'gs://leap-persistent/abbysh/ncar_files/processed'
fco2_recon_dir = 'gs://leap-persistent/your_username'

In [ ]:
def calc_recon_fco2(regridded_members_dir, fco2_recon_dir):
    
    fco2T_path = f'{regridded_members_dir}/pco2_components_202002-202201.zarr'
    print('fco2T path:',fco2T_path)            
    fco2D_path = f"{fco2_recon_dir}/recon_fC02residual_nmse_mon_01x01_202002_202201.zarr"
    print('fco2D path:',fco2D_path)
    file_out = f"{fco2_recon_dir}/recon_pCO2_nmse_mon_01x01_202002-202201.zarr" 
    print('save path:',file_out)

    fco2T_series = xr.open_dataset(fco2T_path,engine='zarr').pco2_T.transpose("time","latitude","longitude") 
    
    fco2_ml_output = xr.open_dataset(fco2D_path,engine='zarr')
    
    fco2D_unseen_series = fco2_ml_output.pco2_residual_unseen.transpose("time","latitude","longitude") 
    fco2D_full_series = fco2_ml_output.pco2_residual_full_recon.transpose("time","latitude","longitude") 
    fco2D_train_series = fco2_ml_output.pco2_residual_train.transpose("time","latitude","longitude") 
    fco2D_test_series = fco2_ml_output.pco2_residual_test.transpose("time","latitude","longitude") 

    fco2T_series = fco2T_series.assign_coords({"time":("time",fco2D_unseen_series.time.data)})
    fco2_unseen = fco2T_series + fco2D_unseen_series    
    fco2_full =  fco2T_series + fco2D_full_series
    fco2_train =  fco2T_series + fco2D_train_series
    fco2_test =  fco2T_series + fco2D_test_series
    
    comp = xr.Dataset({'pco2_recon_unseen':(["time","latitude","longitude"],fco2_unseen.data),     
                    'pco2_recon_full':(["time","latitude","longitude"],fco2_full.data),
                      'pco2_recon_train':(["time","latitude","longitude"],fco2_train.data),
                      'pco2_recon_test':(["time","latitude","longitude"],fco2_test.data)},
                    coords={'time': (['time'],fco2T_series.time.values),
                    'latitude': (['latitude'],fco2T_series.latitude.values),
                    'longitude':(['longitude'],fco2T_series.longitude.values)})
    
    if fs.exists(file_out):
        fs.rm(file_out,recursive=True)
    
    comp = comp.chunk({'time':100,'latitude':45,'longitude':90})
    comp.to_zarr(file_out)
    
    print(f'finished with')

In [ ]:
calc_recon_fco2(regridded_members_dir, fco2_recon_dir)